# Step 0: Project Overview
__In this notebook, we collect raw data for our project:__
- Commodity prices (gold, oil, wheat) via Yahoo Finance (yfinance).
- Kaggle datasets: geopolitical risk index and global news.

All raw files will be stored in the `data/` folder for consistency.

In [ ]:
# 01_data_collection
Fetch commodity prices (gold, oil, wheat) and load Kaggle datasets (geopolitical risk, global news)

# Step 1: Install Dependencies (if missing)
__We install required Python packages if not already available:__
- `yfinance` for fetching financial data.
- `kaggle` for downloading datasets from Kaggle.

Uncomment and run the following lines only if packages are missing.

In [ ]:
# If you don't have yfinance or kaggle installed, run these:
 !pip install yfinance kaggle


# Step 2: Import Libraries and Set Paths
__We import all the necessary libraries and configure global paths:__
- `os` → file system operations.
- `pandas` / `numpy` → data handling.
- `yfinance` → commodity price data.
- `datetime` → date management.

We also define the root directory, create a `data/` folder if it doesn’t exist, and set our analysis date range.

In [14]:
import os
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime

# --- Project root ---
ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))  # notebook assumed in /notebooks
print(f"ROOT: {ROOT}")

# --- Data folders ---
DATA_DIR = os.path.join(ROOT, "data")
RAW_DIR = os.path.join(DATA_DIR, "raw")
PROCESSED_DIR = os.path.join(DATA_DIR, "processed")
MERGED_DIR = os.path.join(DATA_DIR, "merged")

os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(PROCESSED_DIR, exist_ok=True)
os.makedirs(MERGED_DIR, exist_ok=True)

print("ROOT:", ROOT)
print("RAW_DIR:", RAW_DIR)
print("PROCESSED_DIR:", PROCESSED_DIR)

# --- Ensure all exist ---
for folder in [DATA_DIR, RAW_DIR, PROCESSED_DIR, MERGED_DIR]:
    os.makedirs(folder, exist_ok=True)

print("Data folders ready:")
for f in [RAW_DIR, PROCESSED_DIR, MERGED_DIR]:
    print(" -", f)
# choose date range
START = "1985-01-01"
END = None  # None -> yfinance takes up to today


ROOT: C:\Users\taton\PycharmProjects\capstone-data-science
ROOT: C:\Users\taton\PycharmProjects\capstone-data-science
RAW_DIR: C:\Users\taton\PycharmProjects\capstone-data-science\data\raw
PROCESSED_DIR: C:\Users\taton\PycharmProjects\capstone-data-science\data\processed
Data folders ready:
 - C:\Users\taton\PycharmProjects\capstone-data-science\data\raw
 - C:\Users\taton\PycharmProjects\capstone-data-science\data\processed
 - C:\Users\taton\PycharmProjects\capstone-data-science\data\merged


# Step 3: Define Commodity Tickers and Fetch Function
__We define the tickers for gold, WTI crude oil, and wheat futures.__
- Each ticker corresponds to a Yahoo Finance symbol.
- We then implement a helper function `fetch_save()` which:
  1. Downloads the data from Yahoo Finance.
  2. Adds daily returns and 5-day rolling volatility.
  3. Saves the data as a CSV in the `data/` folder.

In [21]:
tickers = {
    "Gold": "GC=F",      # Gold futures
    "WTI":  "CL=F",      # WTI crude oil future
    "Wheat": "ZW=F"      # Wheat futures
}
def add_features(df, price_col="Close"):
    df = df.copy()
    df["Return"] = df[price_col].pct_change()
    df["MA_5"] = df[price_col].rolling(5).mean()
    df["Vol_5"] = df["Return"].rolling(5).std()
    return df

def fetch_save(ticker_symbol, shortname):
    print(f"Downloading {shortname} ({ticker_symbol}) ...")
    df = yf.download(ticker_symbol, start=START, end=END, progress=False, auto_adjust=True)

    if df.empty:
        print(f"⚠️ Warning: empty dataframe for {shortname}")
        return pd.DataFrame()

    # Save raw data
    raw_path = os.path.join(RAW_DIR, f"{shortname.lower()}_raw.csv")
    df.reset_index(inplace=True)
    df.to_csv(raw_path, index=False)
    print(f"✅ Raw {shortname} data saved to {raw_path}")

    # Add features
    df = add_features(df, price_col="Close")

    # Save processed data
    processed_path = os.path.join(PROCESSED_DIR, f"{shortname.lower()}_processed.csv")
    df.to_csv(processed_path, index=False)
    print(f"✅ Processed {shortname} data saved to {processed_path}")

    return df



# Step 4: Fetch and Save Commodity Data
__We use the `fetch_save()` function to download and store data for:__
- Gold futures
- WTI crude oil futures
- Wheat futures

In [22]:
# --- Run for all commodities ---
df_gold = fetch_save(tickers["Gold"], "Gold")
df_wti = fetch_save(tickers["WTI"], "WTI")
df_wheat = fetch_save(tickers["Wheat"], "Wheat")

print("\n📂 Raw data files:", os.listdir(RAW_DIR))
print("📂 Processed data files:", os.listdir(PROCESSED_DIR))

✅ Raw Gold data saved to C:\Users\taton\PycharmProjects\capstone-data-science\data\raw\gold_raw.csv
✅ Processed Gold data saved to C:\Users\taton\PycharmProjects\capstone-data-science\data\processed\gold_processed.csv
✅ Raw WTI data saved to C:\Users\taton\PycharmProjects\capstone-data-science\data\raw\wti_raw.csv
✅ Processed WTI data saved to C:\Users\taton\PycharmProjects\capstone-data-science\data\processed\wti_processed.csv
✅ Raw Wheat data saved to C:\Users\taton\PycharmProjects\capstone-data-science\data\raw\wheat_raw.csv
✅ Processed Wheat data saved to C:\Users\taton\PycharmProjects\capstone-data-science\data\processed\wheat_processed.csv

📂 Raw data files: ['gold_futures.csv', 'gold_futures_raw.csv', 'gold_raw.csv', 'wheat_futures.csv', 'wheat_raw.csv', 'wti_crude.csv', 'wti_raw.csv']
📂 Processed data files: ['gold_processed.csv', 'wheat_processed.csv', 'wti_processed.csv']


# Step 5: Quick Data Inspection
__We verify that the downloaded datasets contain rows and preview the first few entries.__


In [23]:
for name, df in [("Gold", df_gold), ("WTI", df_wti), ("Wheat", df_wheat)]:
    print(name, "rows:", 0 if df is None else len(df))
    display(df.head())


Gold rows: 6302


Price,Date,Close,High,Low,Open,Volume,Return,MA_5,Vol_5
Ticker,,GC=F,GC=F,GC=F,GC=F,GC=F,,,
0,2000-08-30,273.899994,273.899994,273.899994,273.899994,0,NaN,NaN,NaN
1,2000-08-31,278.299988,278.299988,274.799988,274.799988,0,0.016064,NaN,NaN
2,2000-09-01,277.000000,277.000000,277.000000,277.000000,0,-0.004671,NaN,NaN
3,2000-09-05,275.799988,275.799988,275.799988,275.799988,2,-0.004332,NaN,NaN
4,2000-09-06,274.200012,274.200012,274.200012,274.200012,0,-0.005801,275.839996,NaN


WTI rows: 6311


Price,Date,Close,High,Low,Open,Volume,Return,MA_5,Vol_5
Ticker,,CL=F,CL=F,CL=F,CL=F,CL=F,,,
0,2000-08-23,32.049999,32.799999,31.950001,31.950001,79385,NaN,NaN,NaN
1,2000-08-24,31.629999,32.240002,31.400000,31.900000,72978,-0.013105,NaN,NaN
2,2000-08-25,32.049999,32.099998,31.320000,31.700001,44601,0.013279,NaN,NaN
3,2000-08-28,32.869999,32.919998,31.860001,32.040001,46770,0.025585,NaN,NaN
4,2000-08-29,32.720001,33.029999,32.560001,32.820000,49131,-0.004563,32.264,NaN


Wheat rows: 6326


Price,Date,Close,High,Low,Open,Volume,Return,MA_5,Vol_5
Ticker,,ZW=F,ZW=F,ZW=F,ZW=F,ZW=F,,,
0,2000-07-17,244.00,248.0,243.75,248.00,7233,NaN,NaN,NaN
1,2000-07-18,241.25,247.5,241.00,244.25,6523,-0.011270,NaN,NaN
2,2000-07-19,245.00,246.0,239.00,241.25,7136,0.015544,NaN,NaN
3,2000-07-20,247.00,247.5,244.00,244.00,3792,0.008163,NaN,NaN
4,2000-07-21,247.25,248.5,246.50,247.00,4380,0.001012,244.9,NaN


# Step 6: Download Kaggle Datasets
__We now download external datasets from Kaggle for further analysis:__
1. **Gold & Silver Price vs Geopolitical Risk Index**
2. **Global News Dataset**
3. **Gold Price Prediction (LSTM reference dataset)**

⚠️ **Note:**
- This requires you to have your Kaggle API key (`kaggle.json`) set up in `~/.kaggle/`.
- These commands must be run in a notebook or shell with Kaggle configured.

In [24]:
import os

# Make sure we have a "data" directory
os.makedirs("../../data", exist_ok=True)

# 1. Gold & Silver Price vs Geopolitical Risk Index
!kaggle datasets download -d shreyanshdangi/gold-silver-price-vs-geopolitical-risk-19852025 -p data --unzip

# 2. Global News Dataset
!kaggle datasets download -d everydaycodings/global-news-dataset -p data --unzip

# 3. Gold Price Prediction (LSTM reference dataset)
!kaggle kernels pull farzadnekouei/gold-price-prediction-lstm-96-accuracy -p data/lstm_reference



Dataset URL: https://www.kaggle.com/datasets/shreyanshdangi/gold-silver-price-vs-geopolitical-risk-19852025
License(s): Attribution-NonCommercial-NoDerivatives 4.0 International (CC BY-NC-ND 4.0)




  0%|          | 0.00/745k [00:00<?, ?B/s]
100%|##########| 745k/745k [00:00<00:00, 291MB/s]


Dataset URL: https://www.kaggle.com/datasets/everydaycodings/global-news-dataset
License(s): CC0-1.0




  0%|          | 0.00/400M [00:00<?, ?B/s]
 18%|#8        | 72.0M/400M [00:00<00:00, 755MB/s]
 36%|###6      | 144M/400M [00:00<00:00, 611MB/s] 
 51%|#####1    | 204M/400M [00:00<00:00, 555MB/s]
 65%|######5   | 260M/400M [00:00<00:00, 553MB/s]
 79%|#######8  | 314M/400M [00:00<00:00, 537MB/s]
 92%|#########1| 366M/400M [00:00<00:00, 531MB/s]
100%|##########| 400M/400M [00:00<00:00, 544MB/s]


Source code downloaded to data/lstm_reference\gold-price-prediction-lstm-96-accuracy.ipynb


# Step 7: Inspect Downloaded Files & Load External Datasets
__We:__
1. List the contents of the `data/` directory (to check Kaggle download results).
2. Attempt to load:
   - Geopolitical Risk (GPR) dataset
   - Global News dataset

⚠️ Note: Filenames may differ, so adapt accordingly.

In [28]:
# --- Load your original datasets (raw) ---
gpr_path = os.path.join(RAW_DIR, "All_Historical_Data_Separately", "Geopolitical Risk Index Daily.csv")
news_path = os.path.join(RAW_DIR, "Global News dataset", "data.csv")

try:
    gpr = pd.read_csv(gpr_path)
    print("✅ GPR dataset loaded:", gpr.shape)
except FileNotFoundError:
    print("⚠️ GPR dataset not found at:", gpr_path)
    gpr = pd.DataFrame()

try:
    global_news = pd.read_csv(news_path)
    print("✅ Global news dataset loaded:", global_news.shape)
except FileNotFoundError:
    print("⚠️ Global News dataset not found at:", news_path)
    global_news = pd.DataFrame()

# --- Quick sanity checks ---
if not gpr.empty:
    print("\n📊 GPR preview:")
    print(gpr.head(3))

if not global_news.empty:
    print("\n📰 Global news preview:")
    print(global_news.head(3))

✅ GPR dataset loaded: (14861, 5)
✅ Global news dataset loaded: (105375, 12)

📊 GPR preview:
         DATE    GPRD GPRD_ACT  GPRD_THREAT EVENT
0  01-01-1985  230.04   275.20       153.03   NaN
1  02-01-1985  115.68   146.77        87.44   NaN
2  03-01-1985   97.43   158.94        29.46   NaN

📰 Global news preview:
   article_id source_id                   source_name  \
0       89541       NaN  International Business Times   
1       89542       NaN                    Prtimes.jp   
2       89543       NaN                      VOA News   

                                       author  \
0                              Paavan MATHEMA   
1                                         NaN   
2  webdesk@voanews.com (Agence France-Presse)   

                                               title  \
0  UN Chief Urges World To 'Stop The Madness' Of ...   
1              RANDEBOOよりワンランク上の大人っぽさが漂うニットとベストが新登場。   
2  UN Chief Urges World to 'Stop the Madness' of ...   

                                 

# Step 8: Add Basic Features to Commodity Prices
__We define a helper function `add_basic_features()` that:__
- Parses the `Date` column
- Ensures chronological order
- Chooses the correct price column (`Adj Close` if available, otherwise `Close`)
- Computes daily returns, 5-day moving average (MA_5), and 5-day rolling volatility (Vol_5)

Then we apply it to Gold, WTI crude, and Wheat, and save the processed results to CSV.

Feature Engineering for Commodity Prices
__We:__
1. Define `add_basic_features()` to compute:
   - Returns (`pct_change`)
   - 5-day moving average (`MA_5`)
   - 5-day volatility (`Vol_5`)
2. Apply it to Gold, WTI, and Wheat datasets.
3. Save processed outputs as `*_processed.csv`.

In [29]:
def add_basic_features(df):
    df = df.copy()
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df.sort_values('Date', inplace=True)

    # (fix) removed ineffective: df.set_index('Date', inplace=False)
    price_col = 'Adj Close' if 'Adj Close' in df.columns else 'Close'
    df[price_col] = df[price_col].astype(float)

    df['Return'] = df[price_col].pct_change()
    df['MA_5'] = df[price_col].rolling(window=5).mean()
    df['Vol_5'] = df['Return'].rolling(window=5).std()
    return df

gold_feat = add_basic_features(df_gold) if not df_gold.empty else pd.DataFrame()
wti_feat  = add_basic_features(df_wti) if not df_wti.empty else pd.DataFrame()
wheat_feat= add_basic_features(df_wheat) if not df_wheat.empty else pd.DataFrame()

## --- Save processed versions ---
gold_feat.to_csv(os.path.join(PROCESSED_DIR, "gold_processed.csv"), index=False)
wti_feat.to_csv(os.path.join(PROCESSED_DIR, "wti_processed.csv"), index=False)
wheat_feat.to_csv(os.path.join(PROCESSED_DIR, "wheat_processed.csv"), index=False)

print(f"✅ Saved processed files to {PROCESSED_DIR}")
print("Processed files:", os.listdir(PROCESSED_DIR))

print("GPR preview:" if not gpr.empty else "⚠️ GPR empty")
print(gpr.head())



✅ Saved processed files to C:\Users\taton\PycharmProjects\capstone-data-science\data\processed
Processed files: ['gold_processed.csv', 'wheat_processed.csv', 'wti_processed.csv']
GPR preview:
         DATE    GPRD GPRD_ACT  GPRD_THREAT EVENT
0  01-01-1985  230.04   275.20       153.03   NaN
1  02-01-1985  115.68   146.77        87.44   NaN
2  03-01-1985   97.43   158.94        29.46   NaN
3  04-01-1985  157.37   156.88       157.03   NaN
4  05-01-1985   81.36    92.70        77.32   NaN


# Step 11: Quick Check of Saved Processed Files
__We check if our processed CSVs were successfully created and stored in `data/`.__

In [107]:
# quick check of saved files
for fname in ["gold_processed.csv", "wti_processed.csv", "wheat_processed.csv"]:
    print(fname, "->", os.path.exists(os.path.join(DATA_DIR,fname)))


gold_processed.csv -> True
wti_processed.csv -> True
wheat_processed.csv -> True
Price        Date       Close        High         Low        Open Volume  \
Ticker                   GC=F        GC=F        GC=F        GC=F   GC=F   
0      2000-08-30  273.899994  273.899994  273.899994  273.899994      0   
1      2000-08-31  278.299988  278.299988  274.799988  274.799988      0   
2      2000-09-01  277.000000  277.000000  277.000000  277.000000      0   
3      2000-09-05  275.799988  275.799988  275.799988  275.799988      2   
4      2000-09-06  274.200012  274.200012  274.200012  274.200012      0   

Price  Return Vol_5        MA_5  
Ticker                           
0         NaN   NaN         NaN  
1         NaN   NaN         NaN  
2         NaN   NaN         NaN  
3         NaN   NaN         NaN  
4         NaN   NaN  275.839996  


# Step 12: Save Notebook & Commit to Git
__Now that we have processed datasets and a working pipeline, we commit our notebook + CSV files to Git for version control.__

⚠️ Run these commands in the **terminal**, not inside the notebook.

In [ ]:
git add data/*.csv notebooks/01_data_collection.ipynb
git commit -m "Add data collection notebook + initial processed commodity files"
git push origin main


# Step 17: Load Kaggle Datasets into DataFrames
__We:__
1. List available files in `data/` to adapt filenames if needed.
2. Load **Geopolitical Risk Index** and **Global News** datasets.
3. Provide fallback placeholders (`None`) if datasets are missing.

In [19]:
print(gold_feat.index)
print(gold_feat.columns)
print(gold_feat['Date'].dtype)
print(gpr_daily['DATE'].dtype)

NameError: name 'gold_feat' is not defined

# Step 19: Prepare GPR dataset
__We:__
Convert GPR `DATE` column to datetime and resample daily.


In [119]:
# --- Prepare GPR data ---
gpr['DATE'] = pd.to_datetime(gpr['DATE'])
gpr_daily = gpr.set_index('DATE').resample('D').ffill().reset_index()
# Keep only relevant columns
gpr_daily = gpr_daily[['DATE', 'GPRD', 'GPRD_THREAT', 'EVENT']]
print("GPR prepared shape:", gpr_daily.shape)
print(gpr_daily.head())

def prepare_features(df, price_col="Adj_Close", name="Commodity"):
    df = df.copy()
    df["Return"] = df[price_col].pct_change()
    df["MA_5"] = df[price_col].rolling(5).mean()
    df["Vol_5"] = df["Return"].rolling(5).std()
    df = df.dropna(subset=[price_col, "Return", "MA_5", "Vol_5"]).reset_index(drop=True)
    print(f"{name} features prepared:", df.shape)
    return df

# def prepare_feat(df, name):
#     """Make sure features exist and drop NaN rows caused by pct_change/rolling."""
#     df = flatten_columns(df)
#     df = safe_reset(df)# Find best matches
#     possible_cols = df.columns.tolist()
#
#     date_col = 'Date' if 'Date' in possible_cols else df.columns[0]
#     adj_col = [c for c in possible_cols if 'Adj' in c or 'Close' in c][0]
#     return_col = [c for c in possible_cols if 'Return' in c][0]
#     ma_col = [c for c in possible_cols if 'MA' in c][0]
#     vol_col = [c for c in possible_cols if 'Vol' in c][0]
#
#     keep_cols = [date_col, adj_col, return_col, ma_col, vol_col]
#     df = df[keep_cols].dropna().reset_index(drop=True)
#
#     # Rename columns for consistency
#     df.columns = ['Date', 'Adj_Close', 'Return', 'MA_5', 'Vol_5']
#
#     print(f"{name} after feature prep:", df.shape)
#     return df

gold_feat = prepare_features(gold_feat, price_col="Close_GC=F", name="Gold")

wti_feat = prepare_features(wti_feat, price_col="Close_CL=F", name="WTI")

wheat_feat = prepare_features(wheat_feat, price_col="Close_ZW=F", name="Wheat")


GPR prepared shape: (14862, 4)
        DATE    GPRD  GPRD_THREAT EVENT
0 1985-01-02  115.68        87.44   NaN
1 1985-01-03   97.43        29.46   NaN
2 1985-01-04  157.37       157.03   NaN
3 1985-01-05  157.37       157.03   NaN
4 1985-01-06  157.37       157.03   NaN
Gold features prepared: (6283, 9)
WTI features prepared: (6292, 9)
Wheat features prepared: (6306, 9)


# Step 20: Merge All Commodities with Geopolitical Risk Index
__We:__
1. Define utility functions:
   - `flatten_columns()` → handle MultiIndex columns.
   - `safe_reset()` → reset index if needed.
2. Apply preprocessing to Gold, WTI, and Wheat.
3. Merge each with daily GPR data.
4. Display merged dataset previews.

In [120]:
def flatten_columns(df):
    """Flatten MultiIndex columns if necessary."""
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [
            '_'.join([str(c) for c in col if c]).strip()
            for col in df.columns.values
        ]
    if df.index.name == 'Date':
        return df.reset_index()
    return df



# --- Gold ---
gold_feat = flatten_columns(gold_feat)
merged_gold = pd.merge(
    gold_feat,
    gpr_daily,
    left_on='Date',
    right_on='DATE',
    how='left'
)

# --- Oil (WTI) ---
wti_feat = flatten_columns(wti_feat)
merged_wti = pd.merge(
    wti_feat,
    gpr_daily,
    left_on='Date',
    right_on='DATE',
    how='left'
)

# --- Wheat ---
wheat_feat = flatten_columns(wheat_feat)
merged_wheat = pd.merge(
    wheat_feat,
    gpr_daily,
    left_on='Date',
    right_on='DATE',
    how='left'
)

# --- Quick checks ---
print("Gold merged shape:", merged_gold.shape)
print("Oil merged shape:", merged_wti.shape)
print("Wheat merged shape:", merged_wheat.shape)

display(merged_gold.head())
display(merged_wti.head())
display(merged_wheat.head())


Gold merged shape: (6283, 13)
Oil merged shape: (6292, 13)
Wheat merged shape: (6306, 13)


,Date,Close_GC=F,High_GC=F,Low_GC=F,Open_GC=F,Volume_GC=F,Return,Vol_5,MA_5,DATE,GPRD,GPRD_THREAT,EVENT
0,2000-09-07,274.000000,274.000000,274.000000,274.000000,125,-0.000729,0.009121,275.859998,2000-09-07,39.91,44.25,NaN
1,2000-09-08,273.299988,273.299988,273.299988,273.299988,0,-0.002555,0.001991,274.859998,2000-09-08,36.84,35.01,NaN
2,2000-09-11,273.100006,273.100006,273.100006,273.100006,0,-0.000732,0.002235,274.079999,2000-09-11,59.75,66.24,NaN
3,2000-09-12,272.899994,272.899994,272.899994,272.899994,0,-0.000732,0.002209,273.500000,2000-09-12,69.61,84.19,NaN
4,2000-09-13,272.799988,272.799988,272.799988,272.799988,0,-0.000366,0.000871,273.219995,2000-09-13,60.90,88.07,NaN


,Date,Close_CL=F,High_CL=F,Low_CL=F,Open_CL=F,Volume_CL=F,Return,Vol_5,MA_5,DATE,GPRD,GPRD_THREAT,EVENT
0,2000-08-30,33.400002,33.400002,32.099998,32.750000,79214,0.020782,0.016605,32.534,2000-08-30,58.01,60.93,NaN
1,2000-08-31,33.099998,33.700001,32.970001,33.250000,56895,-0.008982,0.015324,32.828,2000-08-31,42.07,32.65,NaN
2,2000-09-01,33.380001,33.450001,32.750000,33.049999,45869,0.008459,0.015155,33.094,2000-09-01,45.34,55.29,NaN
3,2000-09-05,33.799999,33.990002,33.419998,33.950001,55722,0.012582,0.012282,33.280,2000-09-05,52.97,29.37,NaN
4,2000-09-06,34.950001,34.950001,33.830002,33.990002,74692,0.034024,0.015858,33.726,2000-09-06,54.75,72.85,NaN


,Date,Close_ZW=F,High_ZW=F,Low_ZW=F,Open_ZW=F,Volume_ZW=F,Return,Vol_5,MA_5,DATE,GPRD,GPRD_THREAT,EVENT
0,2000-07-24,245.50,249.50,242.50,247.25,4393,-0.007078,0.010935,245.20,2000-07-24,57.74,60.01,NaN
1,2000-07-25,245.50,247.00,243.00,244.75,5973,0.000000,0.008619,246.05,2000-07-25,50.43,26.21,NaN
2,2000-07-26,244.25,247.50,243.00,245.50,3755,-0.005092,0.005956,245.90,2000-07-26,39.00,37.07,NaN
3,2000-07-27,245.75,247.75,244.25,244.25,5397,0.006141,0.005238,245.65,2000-07-27,74.02,79.14,NaN
4,2000-07-28,248.50,252.00,246.00,246.00,5698,0.011190,0.007642,245.90,2000-07-28,38.83,49.68,NaN


# Step 21 — Save merged datasets & Quick check of saved files (Notebook)

In [115]:
merged_gold.to_csv(os.path.join(DATA_DIR, "gold_merged.csv"), index=False)
merged_wti.to_csv(os.path.join(DATA_DIR, "wti_merged.csv"), index=False)
merged_wheat.to_csv(os.path.join(DATA_DIR, "wheat_merged.csv"), index=False)

print("Merged datasets saved in:", DATA_DIR)
# Quick check of saved processed files
for fname in ["gold_processed.csv", "wti_processed.csv", "wheat_processed.csv"]:
    exists = os.path.exists(os.path.join(DATA_DIR, fname))
    print(f"{fname} -> {'✅ exists' if exists else '❌ missing'}")

Merged datasets saved in: data
gold_processed.csv -> ✅ exists
wti_processed.csv -> ✅ exists
wheat_processed.csv -> ✅ exists


# Step 22 — Save to Git (Terminal, not notebook)

In [68]:
git add notebooks/01_data_collection.ipynb notebooks/notebooks/data/*.csv .gitignore
git commit -m "Add data collection notebook + initial processed commodity files"
git push origin main


SyntaxError: invalid decimal literal (495343877.py, line 1)

In [117]:
print(gold_feat[['Return', 'Vol_5']].info())
print(gold_feat[['Return', 'Vol_5']].head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6288 entries, 0 to 6287
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Return  0 non-null      float64
 1   Vol_5   0 non-null      float64
dtypes: float64(2)
memory usage: 98.4 KB
None
   Return  Vol_5
0     NaN    NaN
1     NaN    NaN
2     NaN    NaN
3     NaN    NaN
4     NaN    NaN


In [118]:
print(merged_gold.columns)
print(merged_gold[['Return', 'Vol_5']].head())

Index(['Date', 'Close_GC=F', 'High_GC=F', 'Low_GC=F', 'Open_GC=F',
       'Volume_GC=F', 'Return', 'Vol_5', 'MA_5', 'DATE', 'GPRD', 'GPRD_THREAT',
       'EVENT'],
      dtype='object')
   Return  Vol_5
0     NaN    NaN
1     NaN    NaN
2     NaN    NaN
3     NaN    NaN
4     NaN    NaN
